In [10]:
import copy
import random
import operator
import re
import math
import gensim

import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns

from functools import reduce
from collections import Counter

# % matplotlib inline
# sns.set()

In [11]:
stopwords = [
    "i",          "me",         "my",         "myself",     "we",        
    "our",        "ours",       "ourselves",  "you",        "your",      
    "yours",      "yourself",   "yourselves", "he",         "him",       
    "his",        "himself",    "she",        "her",        "hers",      
    "herself",    "it",         "its",        "itself",     "they",      
    "them",       "their",      "theirs",     "themselves", "what",      
    "which",      "who",        "whom",       "this",       "that",      
    "these",      "those",      "am",         "is",         "are",       
    "was",        "were",       "be",         "been",       "being",     
    "have",       "has",        "had",        "having",     "do",        
    "does",       "did",        "doing",      "would",      "should",    
    "could",      "ought",      "i'm",        "you're",     "he's",      
    "she's",      "it's",       "we're",      "they're",    "i've",      
    "you've",     "we've",      "they've",    "i'd",        "you'd",     
    "he'd",       "she'd",      "we'd",       "they'd",     "i'll",      
    "you'll",     "he'll",      "she'll",     "we'll",      "they'll",   
    "isn't",      "aren't",     "wasn't",     "weren't",    "hasn't",    
    "haven't",    "hadn't",     "doesn't",    "don't",      "didn't",    
    "won't",      "wouldn't",   "shan't",     "shouldn't",  "can't",     
    "cannot",     "couldn't",   "mustn't",    "let's",      "that's",    
    "who's",      "what's",     "here's",     "there's",    "when's",    
    "where's",    "why's",      "how's",      "a",          "an",        
    "the",        "and",        "but",        "if",         "or",        
    "because",    "as",         "until",      "while",      "of",        
    "at",         "by",         "for",        "with",       "about",     
    "against",    "between",    "into",       "through",    "during",    
    "before",     "after",      "above",      "below",      "to",        
    "from",       "up",         "down",       "in",         "out",       
    "on",         "off",        "over",       "under",      "again",     
    "further",    "then",       "once",       "here",       "there",     
    "when",       "where",      "why",        "how",        "all",       
    "any",        "both",       "each",       "few",        "more",      
    "most",       "other",      "some",       "such",       "no",        
    "nor",        "not",        "only",       "own",        "same",      
    "so",         "than",       "too",        "very",       "will"
]

In [12]:
def single(string):
    li = list(string)
    i =[0]+list(filter(lambda x:li[x]!=li[x-1],range(1,len(li))))
    return ''.join([li[x] for x in i])
def clearstring(string):
    string = re.sub('<sssss>' , ' ', string)
    string = re.sub('- *[lr]rb *-', ' ', string)
    string = re.sub('(^| )http:[^ ]+', ' ', string)
    string = re.sub('(^| )[A-Za-z]( |$)', ' ', string)
    string = re.sub('(^| )\'[^ ]+', ' ', string)
    string = re.sub('(^| )n\'t ', ' ', string)
    string = re.sub('[^A-Za-z !?/]+', '', string)
    string = re.sub('(^| )[A-Za-z]( |$)', ' ', string)
    string = re.sub('/', ' ', string)
#     string = string.split()
#     # string = set(string) - set(stopwords)
#     _str = []
#     for x in string:
#         if len(x) > 7:
#             x = single(x)
#         if x not in stopwords: 
#             _str += [x]
#     # string = [single(x) for x in string if x not in stopwords]
#     string = ' '.join(_str)
    return string

In [13]:
def readDatacont(fpath):
    label = []
    Dataset = [] 
    table = {'LOW':{'count': Counter(), 'word_id':{}}, 
             'MID':{'count': Counter(), 'word_id':{}}, 
             'HIG':{'count': Counter(), 'word_id':{}}}
    file = open(fpath, encoding='UTF-8')
    i = 0
    for line in file:
        l = line.split('\t\t')
        label.append(l[0])
        words = clearstring(l[1].strip()).split()
        for wd in words:
            try:
                table[l[0]]['word_id'][wd] += [i]
            except:
                table[l[0]]['word_id'][wd] = [i]
        words_count = Counter(words)
        table[l[0]]['count'] += words_count
        Dataset.append(words_count)
        i += 1
    file.close()
    return label, Dataset, table
def readTest(fpath):
    Dataset = [] 
    file = open(fpath, encoding='UTF-8')
    for line in file:
        l = line.split('\t\t')
        words = Counter(clearstring(l[1].strip()).split())
        Dataset.append(words)
    file.close()
    return Dataset

In [14]:
label, train_set, table = readDatacont('MulLabelTrain.ss')
test_set = readTest('MulLabelTest.ss')
vlabel, valid_set, _ = readDatacont('SmallValid.ss')

In [15]:
all_counter = table['MID']['count'] + table['LOW']['count'] + table['HIG']['count']

In [16]:
l = sorted(all_counter.items(),key=lambda x:x[1], reverse=True)
l[:100]

[('the', 502062),
 ('and', 323406),
 ('to', 215867),
 ('it', 166285),
 ('of', 160553),
 ('was', 159909),
 ('is', 128791),
 ('for', 111967),
 ('in', 102481),
 ('that', 96402),
 ('with', 87041),
 ('but', 85088),
 ('you', 85059),
 ('my', 78769),
 ('this', 72535),
 ('we', 70186),
 ('!', 69769),
 ('on', 69676),
 ('they', 68000),
 ('have', 65428),
 ('not', 56457),
 ('had', 55599),
 ('so', 53125),
 ('good', 53084),
 ('are', 50170),
 ('place', 48271),
 ('at', 47682),
 ('were', 47175),
 ('food', 44492),
 ('as', 42529),
 ('be', 40846),
 ('there', 40294),
 ('like', 39644),
 ('just', 35152),
 ('great', 34475),
 ('here', 34472),
 ('all', 34339),
 ('out', 32946),
 ('if', 32888),
 ('do', 31990),
 ('me', 31855),
 ('very', 30491),
 ('one', 29872),
 ('or', 29735),
 ('their', 28536),
 ('really', 27968),
 ('get', 27859),
 ('some', 26344),
 ('our', 25398),
 ('time', 25322),
 ('from', 24949),
 ('up', 24886),
 ('go', 24699),
 ('would', 24415),
 ('when', 24110),
 ('which', 23792),
 ('about', 23719),
 ('back',

In [17]:
LMH = {}
for key in all_counter.keys():
    count = [table['LOW']['count'][key], table['MID']['count'][key], table['HIG']['count'][key]]
    count = np.array(count)
    LMH[key] = {
        'count': count,
        'importance': count.var()/count.sum()
    }
l = sorted(LMH.items(), key=lambda x:x[1]['importance'], reverse=True)
l[:100]

[('the',
  {'count': array([173927, 201810, 126325]), 'importance': 1934.554634022624}),
 ('was',
  {'count': array([65404, 61940, 32565]), 'importance': 1357.2231998615878}),
 ('and',
  {'count': array([101973, 133384,  88049]),
   'importance': 1111.7057960169777}),
 ('to',
  {'count': array([78717, 83398, 53752]), 'importance': 784.4584870416609}),
 ('!',
  {'count': array([13125, 30425, 26219]), 'importance': 777.857838804563}),
 ('but',
  {'count': array([34014, 33851, 17223]), 'importance': 729.2510681751703}),
 ('it',
  {'count': array([60022, 65891, 40372]), 'importance': 716.1646984928819}),
 ('not',
  {'count': array([25140, 20218, 11099]), 'importance': 599.3389777470759}),
 ('good',
  {'count': array([18282, 24212, 10590]), 'importance': 585.8443389512637}),
 ('of',
  {'count': array([53680, 64888, 41985]), 'importance': 544.6036151440472}),
 ('for',
  {'count': array([39264, 45282, 27421]), 'importance': 491.70072332820286}),
 ('were',
  {'count': array([19705, 18084,  938

In [34]:
sus_sample = [0]*len(train_set)
name = {0:'LOW',1:'MID',2:'HIG'}
for wd in l[:7000]:
    label_i = name[wd[1]['count'].argmin()]
#     print(label_i)
#     print(wd[0])
    word_ = table[label_i]['word_id']
    if wd[0] in word_:
        for sid in set(word_[wd[0]]):
            sus_sample[sid] += 1

HIG
the
HIG
was
HIG
and
HIG
to
LOW
!
HIG
but
HIG
it
HIG
not
HIG
good
HIG
of
HIG
for
HIG
were
HIG
that
LOW
great
HIG
is
HIG
with
HIG
we
HIG
on
HIG
very
HIG
would
HIG
did
HIG
in
HIG
they
HIG
like
HIG
had
HIG
ok
HIG
pretty
LOW
delicious
HIG
at
HIG
food
LOW
love
LOW
you
HIG
no
HIG
really
HIG
be
HIG
my
HIG
are
HIG
have
HIG
there
HIG
little
LOW
amazing
HIG
bland
HIG
?
HIG
nothing
HIG
nice
HIG
bad
HIG
she
HIG
minutes
HIG
our
HIG
just
HIG
so
HIG
decent
HIG
some
HIG
which
HIG
ordered
HIG
place
HIG
do
HIG
if
HIG
maybe
HIG
me
HIG
out
HIG
okay
HIG
as
HIG
asked
LOW
always
LOW
perfect
HIG
or
LOW
definitely
LOW
favorite
HIG
lunch
LOW
also
HIG
better
HIG
up
HIG
back
HIG
here
HIG
bit
HIG
mediocre
HIG
tasty
HIG
when
HIG
get
HIG
only
HIG
came
HIG
their
HIG
more
HIG
time
HIG
happy
HIG
service
LOW
awesome
HIG
too
LOW
fresh
HIG
bar
HIG
this
HIG
because
HIG
average
HIG
said
HIG
seemed
LOW
can
HIG
however
HIG
other
HIG
waitress
HIG
table
HIG
server
HIG
got
HIG
order
LOW
friendly
HIG
after
HIG
dry
HIG
chicken


HIG
handed
LOW
strip
HIG
restaurants
HIG
enchilada
HIG
college
HIG
spice
HIG
frustrating
HIG
kids
HIG
enjoyable
HIG
comes
HIG
hell
HIG
happen
HIG
eggs
LOW
coconut
HIG
popular
LOW
dog
HIG
flagged
LOW
sundays
LOW
mouth
HIG
vietnamese
HIG
margarita
HIG
tab
HIG
glasses
HIG
behind
HIG
checks
HIG
sign
HIG
dip
HIG
across
HIG
basic
HIG
ham
HIG
groupon
HIG
surprisingly
HIG
north
HIG
orange
HIG
improve
HIG
fatty
HIG
memorable
LOW
plan
HIG
terminal
HIG
id
HIG
wraps
HIG
missing
HIG
cons
LOW
books
LOW
relaxing
HIG
sick
LOW
selections
HIG
excuse
HIG
relatively
HIG
joint
HIG
idea
HIG
spring
HIG
pad
LOW
goods
LOW
satisfied
LOW
diverse
HIG
checking
MID
microwaved
HIG
nearly
HIG
jalapeno
LOW
scrumptious
HIG
minus
HIG
cash
LOW
absolute
HIG
inconsistent
LOW
smoked
HIG
attempt
HIG
working
HIG
mild
LOW
treats
HIG
bunch
HIG
higher
LOW
daily
HIG
today
HIG
passable
LOW
mushrooms
HIG
handle
HIG
acted
HIG
women
LOW
weeks
HIG
interior
HIG
wants
LOW
rib
HIG
brisket
HIG
entrees
LOW
nutella
HIG
mills
HIG
fault
LOW
s

break
LOW
mouthwatering
HIG
corrected
HIG
reheated
LOW
collection
HIG
noted
HIG
perplexed
LOW
lolololo
HIG
teriyaki
HIG
additional
HIG
nickel
HIG
awkwardly
HIG
apathetic
MID
toyama
HIG
boiled
LOW
paletas
HIG
dunno
LOW
asparagus
LOW
camelback
HIG
uninspiring
HIG
young
LOW
recommendation
HIG
fundido
HIG
peanuts
LOW
truffles
HIG
promising
HIG
rated
HIG
kill
HIG
eel
HIG
listed
HIG
booth
LOW
finest
HIG
quiet
HIG
marginal
LOW
patient
HIG
warrant
HIG
breading
LOW
films
HIG
rid
LOW
keeps
HIG
warmer
HIG
managed
HIG
hmmm
LOW
magical
HIG
inform
HIG
plaza
LOW
botanical
HIG
important
HIG
plopped
HIG
boxes
LOW
melt
LOW
brussel
HIG
air
LOW
robert
LOW
woohoo
HIG
stink
LOW
plants
HIG
dump
HIG
bombay
HIG
comp
HIG
incorrect
HIG
cherry
HIG
suck
LOW
welcome
LOW
sooner
HIG
watched
HIG
campus
HIG
thursday
HIG
albeit
HIG
volume
LOW
sunset
HIG
soaked
HIG
played
LOW
lunchtime
LOW
zucca
HIG
hawaiian
HIG
na
HIG
dressed
HIG
asu
LOW
mondays
HIG
pot
HIG
blame
HIG
cowboy
HIG
hmm
HIG
cutting
HIG
condescending
HIG
assu

HIG
tilted
HIG
multiple
HIG
student
HIG
killed
LOW
connected
HIG
miffed
LOW
eaten
LOW
duo
HIG
speakers
HIG
bush
HIG
kyoto
HIG
uncooked
MID
douchebag
HIG
rolled
HIG
soupy
HIG
bathrooms
HIG
faux
LOW
owned
LOW
blueberries
HIG
male
LOW
pear
LOW
workout
HIG
amateur
MID
udupi
LOW
lit
LOW
distance
HIG
dragged
LOW
curried
LOW
frequented
HIG
bjs
LOW
tube
LOW
rootbeer
LOW
seasons
HIG
tikka
HIG
ends
HIG
strips
LOW
colors
HIG
smallest
HIG
yc
HIG
comprehend
HIG
careless
MID
downgrading
MID
gag
HIG
rosie
HIG
trick
HIG
merely
LOW
mountains
HIG
communication
HIG
centrally
HIG
shocked
HIG
spotted
HIG
orlando
HIG
hm
LOW
maurizio
HIG
promoter
HIG
thimble
HIG
reeked
HIG
seamed
MID
boxer
LOW
vacation
HIG
miso
HIG
stare
LOW
donut
LOW
kha
HIG
texas
HIG
cityscape
LOW
ending
LOW
shea
LOW
jalepeno
LOW
shochu
LOW
brulee
HIG
using
HIG
im
HIG
rio
LOW
gps
HIG
upcharge
HIG
lupe
LOW
em
HIG
dose
HIG
skins
LOW
float
LOW
wedding
HIG
result
HIG
direction
LOW
foie
LOW
midday
LOW
stacks
LOW
bonnie
LOW
cotta
HIG
chunky
HIG


document
MID
dolphins
MID
scathing
MID
duped
MID
serverdude
MID
contaminated
MID
retrain
MID
isan
MID
chucky
MID
tolerated
MID
doublex
MID
dotcom
LOW
taook
LOW
quando
LOW
wexler
LOW
pmam
LOW
carole
LOW
banofi
LOW
comma
LOW
pasa
LOW
guyana
LOW
arrachera
LOW
deborah
LOW
claudio
LOW
gnudi
LOW
marla
LOW
chele
LOW
gcc
LOW
pepperpot
HIG
cause
HIG
harder
MID
umm
HIG
nevertheless
HIG
occasions
LOW
adore
HIG
claim
HIG
search
HIG
responsible
HIG
squeeze
HIG
purse
LOW
tai
HIG
banh
HIG
content
HIG
amber
HIG
helping
HIG
jerk
MID
lard
HIG
advised
HIG
assuming
HIG
thaiger
HIG
brings
HIG
pbr
LOW
elusive
HIG
giants
LOW
customize
LOW
grandma
HIG
douchebags
LOW
believer
HIG
holes
HIG
relief
LOW
difficulty
LOW
chris
MID
simply
LOW
glorious
HIG
bust
HIG
wound
HIG
risk
HIG
closing
HIG
airports
HIG
board
HIG
sideways
HIG
disdain
LOW
playful
HIG
brag
HIG
nourish
LOW
church
HIG
suffered
HIG
geared
HIG
barrel
LOW
liqueur
HIG
hers
LOW
coyle
LOW
sommelier
HIG
applebees
HIG
cigarette
LOW
popovers
HIG
awfully
LOW
v

HIG
stinky
LOW
expansion
HIG
regardless
LOW
murals
HIG
nonchain
HIG
condiment
HIG
stating
HIG
bombs
HIG
katsu
LOW
justice
LOW
!!!!!!
LOW
healthiest
HIG
cfu
HIG
consensus
LOW
condesa
LOW
meltinyourmouth
HIG
biscuit
LOW
december
LOW
deserts
HIG
brats
LOW
butterfly
HIG
inventive
LOW
piping
LOW
jidori
HIG
tomatillo
HIG
venture
HIG
speaking
LOW
meyer
LOW
rocky
HIG
naturally
LOW
cares
LOW
tres
HIG
anthem
LOW
sauteed
LOW
com
LOW
creamiest
LOW
perry
LOW
pebbles
HIG
eek
HIG
glutenfree
HIG
online
HIG
bialy
HIG
segment
HIG
printing
HIG
unruly
HIG
kaiser
HIG
revenue
HIG
tweaks
HIG
acknowledgement
HIG
preoccupied
LOW
exquisitely
HIG
krishna
LOW
devouring
LOW
wellexecuted
LOW
infusions
HIG
bumper
LOW
taboule
HIG
sunup
LOW
thyme
LOW
sonny
HIG
taberna
LOW
google
HIG
snatched
LOW
blossoms
LOW
intoxicating
LOW
vouch
LOW
venison
HIG
accounts
MID
principle
MID
atl
HIG
banchan
HIG
pours
HIG
recall
HIG
ads
LOW
photography
HIG
fate
HIG
ihop
MID
affliction
HIG
irritates
LOW
yummo
LOW
map
LOW
invention
HIG
com

riverview
HIG
eggroll
LOW
riga
HIG
tackle
LOW
bookstores
HIG
masked
HIG
wob
LOW
nordy
HIG
ikura
HIG
uncool
LOW
buzzed
HIG
avenue
LOW
convert
HIG
clothing
HIG
cornhole
LOW
omfg
HIG
disclaimer
LOW
pastor
LOW
rent
LOW
charleen
LOW
courtyard
HIG
boobs
HIG
variations
LOW
brainer
LOW
jeremy
LOW
licking
HIG
shaker
LOW
cheba
HIG
hi
HIG
guests
HIG
heels
HIG
mildly
HIG
sweating
HIG
crinkle
HIG
corona
HIG
prize
HIG
spears
LOW
performances
LOW
applesauce
MID
disappearing
HIG
chalk
LOW
riding
HIG
certificates
HIG
slutty
LOW
gingerbread
HIG
kobalt
LOW
jw
HIG
chalked
HIG
mbfgr
LOW
thanh
HIG
griddle
HIG
bibimbap
HIG
pedestrians
LOW
friendship
HIG
flamingo
HIG
intentional
HIG
cabinets
HIG
goers
LOW
resturants
MID
syndrome
LOW
delicacy
MID
waaaaay
LOW
inn
LOW
apache
MID
overrun
HIG
duh
HIG
gal
HIG
chilis
MID
begin
HIG
surely
HIG
stored
LOW
resisted
HIG
flooring
LOW
ecstasy
HIG
confirmed
HIG
diva
HIG
understandably
HIG
communicate
HIG
tsk
HIG
danger
LOW
alla
HIG
blu
HIG
piccata
HIG
logan
LOW
exploring
LO

In [63]:
sus_alpha = 30
sample_remain = list(filter(lambda x: x[1]<sus_alpha, zip(range(len(train_set)), sus_sample)))

In [64]:
sample_remain_id = list(map(lambda x: x[0], sample_remain))
new_trainset = list(map(lambda x: train_set[x],sample_remain_id))
new_label = list(map(lambda x:label[x], sample_remain_id))

In [43]:
new_trainset[:100]

[Counter({'?': 1,
          'all': 1,
          'already': 1,
          'am': 1,
          'and': 8,
          'around': 4,
          'at': 1,
          'avocado': 1,
          'away': 1,
          'back': 1,
          'baked': 1,
          'be': 2,
          'because': 1,
          'been': 1,
          'bo': 1,
          'boy': 1,
          'boyfriend': 1,
          'breaded': 1,
          'breakfast': 2,
          'breast': 1,
          'brick': 1,
          'burrito': 2,
          'cajun': 1,
          'came': 4,
          'carne': 2,
          'chicken': 3,
          'coffee': 2,
          'come': 1,
          'commented': 1,
          'day': 1,
          'eggs': 2,
          'employee': 1,
          'even': 1,
          'fat': 1,
          'finally': 1,
          'food': 1,
          'front': 1,
          'good': 2,
          'got': 1,
          'green': 1,
          'had': 2,
          'half': 1,
          'hand': 1,
          'have': 1,
          'how': 1,
          'in': 1,
   

In [44]:
file = open('wordvec_6B_50d.txt', encoding='UTF-8')
vec, wdvec_dict, order = [], {}, 0
for line in file:
    item = line.strip().split()
    vec.append(list(map(float,item[1:])))
    wdvec_dict[item[0]] = order
    order += 1
file.close()
def w2v(word):
    if (word in wdvec_dict):
        return np.array(vec[wdvec_dict[word]])
    return np.zeros(50)

In [65]:
train_x = []
for ts in new_trainset:
    v = np.array(list(map(w2v,ts.keys())))
    if not v.any():
        v = np.zeros((1,50))
    train_x.append(v.mean(axis=0))
train_x = np.array(train_x)

test_x = []
for ts in test_set:
    v = np.array(list(map(w2v,ts.keys())))
    if not v.any():
        v = np.zeros((1,50))
    test_x.append(v.mean(axis=0))
test_x = np.array(test_x)
                 
valid_x = []
for ts in valid_set:
    v = np.array(list(map(w2v,ts.keys())))
    if not v.any():
        v = np.zeros((1,50))
    valid_x.append(v.mean(axis=0))
valid_x = np.array(valid_x)

In [66]:
x0 = np.ones((train_x.shape[0],1))
tr_x = np.concatenate((x0, train_x), axis=1)
x0 = np.ones((valid_x.shape[0], 1))
va_x = np.concatenate((x0, valid_x), axis=1)
x0 = np.ones((test_x.shape[0], 1))
te_x = np.concatenate((x0, test_x), axis=1)

In [55]:
def evaluate(TP, FN, TN, FP):
	Accuracy = (TP + TN) / np.float64(TP + FP + TN + FN)
	return Accuracy
def check(res, std):
	count = [0]*4
	for r, s in zip(res, std):
		wrong = int(r != s)
		neg = int(s != 1)
		count[neg * 2 + wrong] += 1
	print(evaluate(*count))
	return 
def sigmoid(X):
	return 1 / (1 + np.exp(-X))
class LogicalRegression(object):
	"""docstring for LogicalRegression"""
	def __init__(self, train_X, train_y, weights=None, times=1000, alpha=0.01, c=2, style='batch'):
		self.train_X = train_X
		self.train_y = train_y
		self.times = times
		self.alpha = alpha
		self.c = c
		gradient = []
		if weights == None:
			weights = np.zeros((1,train_X.shape[1])) # 1*M
		# alpha = 0.000001
		epsilon = 1e-6
		self.w0 = weights.T # M*1
		p = sigmoid(np.dot(train_X, self.w0)) # N*M X M*1 = N*1
		J0 = self.logistic_cost(p, train_y)
		K0 = K00 = J0
		for t in range(times):
			p_y = p - train_y
			w = self.w0 - alpha * np.dot(train_X.T, p_y) 
			# print(np.dot(train_X.transpose(), p_y))
			gradient.append(np.sum(-np.dot(train_X.T, p_y)))
			# w = self.w0 - alpha * np.dot(train_X.transpose(), p_y) / len(p) - lambda_ * self.w0 / len(p) #40*1
			p = sigmoid(np.dot(train_X, w))
			J = self.logistic_cost(p, train_y)
			if t > 7 and K00 == J:
				break
			K00 = K0
			K0 = J
			if J < J0: # not convergent
				J0 = J
				self.w0 = w
			else:
				alpha /= c
				if np.linalg.norm(w - self.w0) < epsilon * np.linalg.norm(self.w0):
					# print('times: ', t)
					break
		return # M*1
	def LR(self, train_X, train_y,weights=None):
		gradient = []
		if weights == None:
			weights = np.zeros((1,train_X.shape[1])) # 1*M
		# alpha = 0.000001
		epsilon = 1e-3
		w0 = weights.T # M*1
		# print(w0)
		p = sigmoid(np.dot(train_X, w0)) # N*M X M*1 = N*1
		J0 = self.logistic_cost(p, train_y)
		K0 = K00 = J0
		for t in range(self.times):
			p_y = p - train_y
			w = w0 - self.alpha * np.dot(train_X.T, p_y) 
			# print(np.dot(train_X.transpose(), p_y))
			gradient.append(np.sum(-np.dot(train_X.T, p_y)))
			# w = w0 - alpha * np.dot(train_X.transpose(), p_y) / len(p) - lambda_ * w0 / len(p) #40*1
			p = sigmoid(np.dot(train_X, w))
			J = self.logistic_cost(p, train_y)
			if t > 7 and K00 == J:
				break
			K00 = K0
			K0 = J
			if J < J0: # not convergent
				J0 = J
				w0 = w
			else:
				self.alpha /= self.c
				if np.linalg.norm(w - w0) < epsilon * np.linalg.norm(w0):
					# print('times: ', t)
					break
		return w0 # M*1
	def logistic_cost(self, p, train_y):
		_p = copy.deepcopy(p)
		_p[train_y == 0] = 1 - _p[train_y == 0]
		return -sum(np.log(_p+0.000001)) 
	def predict(self, test_X):
		return  sigmoid(np.dot(test_X, self.w0))
	def groupValidation(self, k=10):
		n = self.train_X.shape[0]
		m = math.ceil(n/k)
		n_id = list(range(n))
		random.shuffle(n_id)
		L = []
		L += list(map(lambda x:n_id[x:x+m] ,map(lambda i:i*m, range(10))))
		return L
	def crossValidation(self):
		L = self.groupValidation()
		for i in range(len(L)):
			cp_L = copy.deepcopy(L)
			vali_id = cp_L.pop(i)
			vali_X, vali_y = self.train_X[vali_id], self.train_y[vali_id]
			tra_x, tra_y = self.train_X[reduce(operator.add,cp_L)], self.train_y[reduce(operator.add,cp_L)]
			weights = self.LR(tra_x, tra_y)
			res = np.dot(vali_X, weights)
			res = list(map(lambda x: 1 if x > 0 else 0, res))
			check(res, vali_y)
		return

In [61]:
def writefile(filename, res):
    file = open(filename,'w')
    for i in res:
        file.write(i+'\n')
    file.close()
    return
def check_label(std, res):
    return (np.array(res) == np.array(std)).sum() / len(std)
def labelTonum(label):
    L_y = list(map(lambda x:1 if x =='LOW' else 0, label))
    L_y = np.array([L_y]).T
    M_y = list(map(lambda x:1 if x =='MID' else 0, label))
    M_y = np.array([M_y]).T
    H_y = list(map(lambda x:1 if x =='HIG' else 0, label))
    H_y = np.array([H_y]).T
    return L_y, M_y, H_y
def numTolabel(y):
    name = ['LOW','MID','HIG']
    return list(map(lambda x:name[x],y))

In [57]:
def scaling(train_X, test_X):
    mean = train_X.mean(axis=0)
    std = train_X.std(axis=0)
    ones1 = np.ones((train_X.shape[0],1))
    ones2 = np.ones((test_X.shape[0],1))
    m1 = np.kron(ones1, mean)
    s1 = np.kron(ones1, std)
    m2 = np.kron(ones2, mean)
    s2 = np.kron(ones2, std)
    r1 = (train_X - m1) / s1
    r2 = (test_X - m2) / s2
    return r1, r2

In [67]:
###################################LR Validtion############################
_,te_x = scaling(tr_x, te_x)
tr_x,va_x = scaling(tr_x, va_x)
L_y, M_y, H_y = labelTonum(new_label)
model_L = LogicalRegression(tr_x, L_y)
model_M = LogicalRegression(tr_x, M_y)
model_H = LogicalRegression(tr_x, H_y)
pred_L = model_L.predict(va_x)
pred_M = model_M.predict(va_x)
pred_H = model_H.predict(va_x)
pred = np.concatenate((pred_L, pred_M, pred_H), axis=1)
###################################LR Validtion############################

c:\users\simon\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.
c:\users\simon\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()
c:\users\simon\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in less


In [68]:
_res = pred.argmax(axis=1)
res = numTolabel(_res)
check_label(res, vlabel)

0.262

In [69]:
res

['LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',


In [9]:
def getset(dataset, voc_id):
    my_set = []
    dataset = set(dataset)
    for line in dataset:
        my_set += [list(map(lambda x: voc_id.index(x), set(line)&set(voc_id)))]
    return my_set
def check_label(res ,std):
    return (np.array(res) == np.array(std)).sum() / len(std)
class onehot(object):
    """docstring for Onehot"""
    def __init__(self, voc_id, label, Train_set, Test_set, sample, K=10):
        self.label = label
        self.pred = []
        if(sample == None):
            sample = len(Train_set)
        Train = []
        Label = []
        for i in sample:
            Train += [Train_set[i]]
            Label += [label[i]]
        for Test_item in Test_set:
            dist= list(map(lambda x:self.calc_distance(x, Test_item), Train))
            NN = sorted(zip(dist, Label), key=lambda x:x[0], reverse=True)
            self.pred += [Counter(list(map(lambda x:x[1], NN[:K]))).most_common(1)[0][0]]
        return 
    def calc_distance(self, a, b):
        epsilon = 0.0001
        c = [x for x in a if x in b]
#         dist = (len(c)*(len(a)+len(b)) / (2*len(a)*len(b)))**4
        dist = (2 * len(c) / (len(a) + len(b)))
        return 1 / (epsilon + dist)

In [10]:
sample = random.sample(range(len(train_set)), 10000)
Train_set = getset(train_set, voc_id)
Valid_set = getset(valid_set, voc_id)
model = onehot(voc_id, label, Train_set, Valid_set, sample)

TypeError: unhashable type: 'Counter'